In [1]:
import pandas as pd
import numpy as np

In [2]:
from bs4 import BeautifulSoup
import requests

In [3]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(url, "html.parser")
soup.title

<title>List of postal codes of Canada: M - Wikipedia</title>

In [4]:
import csv

In [6]:
table =soup.find("table", class_ = "wikitable sortable")

output_rows = []
for table_row in table.findAll('tr'):
    columns = table_row.findAll('td')
    output_row = []
    for column in columns:
        output_row.append(column.text)
    output_rows.append(output_row)
    
with open('wikipedia.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(output_rows)

In [7]:
df = pd.read_csv('wikipedia.csv', header=None)
header = ['Postalcode', 'Borough', 'Neighborhood']
df.columns = header
df = df.replace(['\r', '\n'],'', regex=True)
df.head(10)

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [8]:
df.shape

(180, 3)

In [33]:
#part 1
df = df[df["Borough"] != "Not assigned"]
df.shape

(103, 5)

In [10]:
df = df.groupby(['Postalcode','Borough']).Neighborhood.apply(lambda x: ', '.join(x)).to_frame()
df.reset_index(inplace=True)
df.head(10)

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [22]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_f11589defb2d408c947c9108a6bedcbc = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='3Nc3OSXZBHHuQAOQJxOWcaRCpWaFXGzXSPNrWGx9E2Vz',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_f11589defb2d408c947c9108a6bedcbc.get_object(Bucket='capstoneweek1-donotdelete-pr-sbxghygsxctgfd',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ 
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [30]:
df['Latitude'] = df_data_1['Latitude']
df['Longitude'] = df_data_1['Longitude']

In [32]:
#part 2
df.head(10)

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [34]:
df_toronto = df[df['Borough'].str.contains("Toronto")]
df_toronto.shape

(39, 5)

In [36]:
df_toronto.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [37]:

#part three
from geopy.geocoders import Nominatim
import folium
import pandas as pd
import numpy as np

In [38]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.6534817 -79.3839347


In [39]:
#for map
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, label in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [49]:
df_c_toronto = df_toronto[df_toronto['Borough'].str.contains("East")]
df_c_toronto = df_c_toronto.reset_index()
df_c_toronto.head()

,index,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,37,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,43,M4M,East Toronto,Studio District,43.659526,-79.340923
4,87,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558


In [50]:
#lat long east

address = 'East Toronto'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.6261221 -79.3950351


In [51]:
#folium map
map_c_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, label in zip(df_c_toronto['Latitude'], df_c_toronto['Longitude'], df_c_toronto['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_c_toronto)  
    
map_c_toronto

In [41]:
#4square
CLIENT_ID = 'IVQZMQ25XPU1ECLL0XYJTNLBHCXADZVJI12PVRD0RXJLRHAI' 
CLIENT_SECRET = 'DLDV1XYWTGM34CO3HWZBM0AA4PU2Y45AB54T333SNDOSZFPY' 
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: IVQZMQ25XPU1ECLL0XYJTNLBHCXADZVJI12PVRD0RXJLRHAI
CLIENT_SECRET:DLDV1XYWTGM34CO3HWZBM0AA4PU2Y45AB54T333SNDOSZFPY


In [52]:
df_c_toronto.loc[0, 'Neighborhood']

'The Beaches'

In [53]:
neighborhood_latitude = df_c_toronto.loc[0, 'Latitude'] 
neighborhood_longitude = df_c_toronto.loc[0, 'Longitude'] 

neighborhood_name = df_c_toronto.loc[0, 'Neighborhood'] 

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


In [54]:
LIMIT = 50 

radius = 200 

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=IVQZMQ25XPU1ECLL0XYJTNLBHCXADZVJI12PVRD0RXJLRHAI&client_secret=DLDV1XYWTGM34CO3HWZBM0AA4PU2Y45AB54T333SNDOSZFPY&v=20180604&ll=43.67635739999999,-79.2930312&radius=200&limit=50'

In [55]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6005a79ece054c46cc3e1bbe'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 3,
  'suggestedBounds': {'ne': {'lat': 43.67815740179999,
    'lng': -79.2905470857962},
   'sw': {'lat': 43.674557398199994, 'lng': -79.2955153142038}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd461bc77b29c74a07d9282',
       'name': 'Glen Manor Ravine',
       'location': {'address': 'Glen Manor',
        'crossStreet': 'Queen St.',
        'lat': 43.67682094413784,
        'lng': -79.29394208780985,


In [58]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [59]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) 

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,Glen Stewart Park,Park,43.675278,-79.294647
2,Balmy Beach Playground,Playground,43.676078,-79.290805


In [60]:
print('{} venues were returned.'.format(nearby_venues.shape[0]))

3 venues were returned.


In [62]:
def getNearbyVenues(names, latitudes, longitudes, radius=200):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [63]:
east_toronto_venues = getNearbyVenues(names=df_c_toronto['Neighborhood'],
                                   latitudes=df_c_toronto['Latitude'],
                                   longitudes=df_c_toronto['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Business reply mail Processing Centre, South Central Letter Processing Plant Toronto


In [67]:
print(east_toronto_venues.shape)
east_toronto_venues.head()

(23, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,Glen Stewart Park,43.675278,-79.294647,Park
2,The Beaches,43.676357,-79.293031,Balmy Beach Playground,43.676078,-79.290805,Playground
3,"The Danforth West, Riverdale",43.679557,-79.352188,Kalyvia,43.677973,-79.351208,Greek Restaurant
4,"India Bazaar, The Beaches West",43.668999,-79.315572,British Style Fish & Chips,43.668723,-79.317139,Fish & Chips Shop


In [76]:
east_toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",2,2,2,2,2,2
"India Bazaar, The Beaches West",2,2,2,2,2,2
Studio District,15,15,15,15,15,15
The Beaches,3,3,3,3,3,3
"The Danforth West, Riverdale",1,1,1,1,1,1


In [78]:
east_toronto_onehot = pd.get_dummies(east_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

east_toronto_onehot['Neighborhood'] = east_toronto_venues['Neighborhood'] 

fixed_columns = [east_toronto_onehot.columns[-1]] + list(east_toronto_onehot.columns[:-1])
east_toronto_onehot = east_toronto_onehot[fixed_columns]
east_toronto_onehot.head()

,Neighborhood,Auto Workshop,Bakery,Bank,Bar,Bookstore,Brewery,Café,Cheese Shop,Clothing Store,...,Fish & Chips Shop,Gastropub,Greek Restaurant,Ice Cream Shop,Park,Pet Store,Pharmacy,Playground,Seafood Restaurant,Trail
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,"India Bazaar, The Beaches West",0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [79]:
east_toronto_grouped = east_toronto_onehot.groupby('Neighborhood').mean().reset_index()
east_toronto_grouped

,Neighborhood,Auto Workshop,Bakery,Bank,Bar,Bookstore,Brewery,Café,Cheese Shop,Clothing Store,...,Fish & Chips Shop,Gastropub,Greek Restaurant,Ice Cream Shop,Park,Pet Store,Pharmacy,Playground,Seafood Restaurant,Trail
0,"Business reply mail Processing Centre, South C...",0.5,0.000000,0.000000,0.000000,0.000000,0.5,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"India Bazaar, The Beaches West",0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.5,0.000000,0.0,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Studio District,0.0,0.066667,0.066667,0.066667,0.066667,0.0,0.066667,0.066667,0.066667,...,0.0,0.066667,0.0,0.066667,0.000000,0.066667,0.066667,0.000000,0.066667,0.000000
3,The Beaches,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.333333,0.000000,0.000000,0.333333,0.000000,0.333333
4,"The Danforth West, Riverdale",0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [81]:
num_top_venues = 5

for hood in east_toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = east_toronto_grouped[east_toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                venue  freq
0       Auto Workshop   0.5
1             Brewery   0.5
2   Fish & Chips Shop   0.0
3  Seafood Restaurant   0.0
4          Playground   0.0


----India Bazaar, The Beaches West----
                venue  freq
0   Fish & Chips Shop   0.5
1                Park   0.5
2       Auto Workshop   0.0
3  Seafood Restaurant   0.0
4          Playground   0.0


----Studio District----
               venue  freq
0        Coffee Shop  0.13
1  Convenience Store  0.07
2          Pet Store  0.07
3               Bank  0.07
4                Bar  0.07


----The Beaches----
                venue  freq
0               Trail  0.33
1          Playground  0.33
2                Park  0.33
3   Fish & Chips Shop  0.00
4  Seafood Restaurant  0.00


----The Danforth West, Riverdale----
                venue  freq
0    Greek Restaurant   1.0
1       Auto Workshop   0.0
2   Fish & Chips Shop   0.0
3

In [83]:
#top venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [85]:

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = east_toronto_grouped['Neighborhood']

for ind in np.arange(east_toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(east_toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Business reply mail Processing Centre, South C...",Auto Workshop,Brewery,Coffee Shop,Bakery,Bank
1,"India Bazaar, The Beaches West",Park,Fish & Chips Shop,Trail,Clothing Store,Bakery
2,Studio District,Coffee Shop,Convenience Store,Bakery,Pharmacy,Pet Store
3,The Beaches,Trail,Playground,Park,Clothing Store,Bakery
4,"The Danforth West, Riverdale",Greek Restaurant,Trail,Coffee Shop,Bakery,Bank


In [86]:
#cluster
from sklearn.cluster import KMeans

In [89]:
kclusters = 5

east_toronto_grouped_clustering = east_toronto_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(east_toronto_grouped_clustering)

kmeans.labels_[0:10]

array([2, 1, 3, 4, 0], dtype=int32)

In [90]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [91]:
east_toronto_merged = df_c_toronto

east_toronto_merged = east_toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

east_toronto_merged.head()

,index,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,37,M4E,East Toronto,The Beaches,43.676357,-79.293031,4,Trail,Playground,Park,Clothing Store,Bakery
1,41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Trail,Coffee Shop,Bakery,Bank
2,42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,1,Park,Fish & Chips Shop,Trail,Clothing Store,Bakery
3,43,M4M,East Toronto,Studio District,43.659526,-79.340923,3,Coffee Shop,Convenience Store,Bakery,Pharmacy,Pet Store
4,87,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,2,Auto Workshop,Brewery,Coffee Shop,Bakery,Bank


In [96]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [93]:
#east toronto clusters
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(east_toronto_merged['Latitude'], east_toronto_merged['Longitude'], east_toronto_merged['Neighborhood'], east_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters